<a href="https://colab.research.google.com/github/olegkleiman/tf2/blob/master/basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example for basic ML concepts. Demonstrates using @tf.function decorator and GradientTape context 

In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))

print("TensorFlow version: ", tf.__version__)
print("Num GPU Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.gpu_device_name())
# tf.debugging.set_log_device_placement(True)

logdir = './logs/linreg'
writer = tf.summary.create_file_writer(logdir)

slope = 0.4
bias = 1.5
rows = 1000

x_train = tf.random.uniform(shape=(rows,))
perturb = tf.random.normal(shape=(len(x_train),), stddev=0.1)
y_train = slope * x_train + bias + perturb

m = tf.Variable(0.)
b = tf.Variable(0.)


def predict_y_value(x):
    y = m * x + b
    return y


@tf.function
def squared_error(y_pred, y_true):
    # Being decorated with @tf.function, this function will only be parsed (on the first call) into graph.
    # It's actually not executed.
    # The created graph is executed instead.
    # This graph is visualized in TensorBoard with help on tf.summary.* methods
    return tf.reduce_mean(tf.square(y_pred - y_true))


steps = 400
learning_rate = 0.05

# Bracket the loss function call - squared_error()- with
# tf.summary.trace_on() and tf.summary.trace_export()
tf.summary.trace_on(graph=True, profiler=True)

with writer.as_default():
    for epoch in range(steps):
        with tf.GradientTape() as tape:
            predictions = predict_y_value(x_train)
            loss = squared_error(predictions, y_train)
            tf.summary.scalar('loss', loss, step=epoch)

        gradients = tape.gradient(loss, [m, b])
        m.assign_sub(gradients[0] * learning_rate)
        b.assign_sub(gradients[1] * learning_rate)

    tf.summary.trace_export(name='mse_loss',
                            step=0,
                            profiler_outdir=logdir)

print("m: %f, b: %f" % (m.numpy(), b.numpy()))

TensorFlow version:  2.4.1
Num GPU Available:  1
/device:GPU:0
Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
m: 0.429805, b: 1.484098
